# Auto Generated Agent Chat: Multi Task Chats

This notebook showcases how to use a new chat interface of conversable agents in AutoGen: `initiate_chats` to conduct a series of tasks. This new interface allows one to pass multiple tasks and their corresponding dedicated agents.


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

## Example Tasks

In [4]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? """,
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Solve the tasks with a series of chats

In [5]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config={"config_list": config_list},
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config={"config_list": config_list},
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "takeaway_method": "last_msg",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "takeaway_method": "last_msg",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
        },
    ]
)


********************************************************************************
Start work on the following task: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

With the following carryover from previous tasks: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), as well as their performance over the past month in terms of percentage change, we need to access financial data. This can be done using various financial data APIs or Python libraries that provide access to such information.

I will use the `yfinance` 

### Solve the tasks with a series of chats involving group chat



In [6]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    Reply "TERMINATE" in the end when everything is done.
    """,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.
    Reply "TERMINATE" in the end when everything is done.
    """,
    llm_config={"config_list": config_list},
)

groupchat_1 = autogen.GroupChat(agents=[user_proxy, research_assistant, critic], messages=[], max_round=50)

groupchat_2 = autogen.GroupChat(agents=[user_proxy, writer, critic], messages=[], max_round=50)

manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)
manager_2 = autogen.GroupChatManager(
    groupchat=groupchat_2,
    name="Writing_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
user.initiate_chats(
    [
        {"recipient": research_assistant, "message": financial_tasks[0], "takeaway_method": "last_msg"},
        {"recipient": manager_1, "message": financial_tasks[1], "takeaway_method": "llm"},
        {"recipient": manager_2, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Start work on the following task: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

With the following carryover from previous tasks: 


********************************************************************************
User (to Researcher):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

--------------------------------------------------------------------------------
Researcher (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), as well as their performance over the past month in terms of percentage change, we need to access financial data. This can be done using various financial data APIs or Python libraries that provide access to such information.

I will use the `yfinance` library in Python,